# Import the necessary libraries, read csv file, merge movies and credits table

In [2]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import re
import warnings
import operator
import collections
! pip install rake_nltk
from rake_nltk import Rake
warnings.filterwarnings("ignore", category=DeprecationWarning)
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brajandangllia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brajandangllia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [5]:
credits.columns=['id','title','cast','crew']
movies_merge=movies.merge(credits, on='id')

In [6]:
movies_merge.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [7]:
movies_df= pd.DataFrame(movies_merge[['id','popularity','keywords','homepage','original_title','budget','revenue','genres', 'overview', 'cast', 'tagline']])
movies_df.head(3)

,id,popularity,keywords,homepage,original_title,budget,revenue,genres,overview,cast,tagline
0,19995,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",http://www.avatarmovie.com/,Avatar,237000000,2787965087,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",Enter the World of Pandora.
1,285,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",http://disney.go.com/disneypictures/pirates/,Pirates of the Caribbean: At World's End,300000000,961000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","At the end of the world, the adventure begins."
2,206647,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",http://www.sonypictures.com/movies/spectre/,Spectre,245000000,880674609,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",A Plan No One Escapes


In [8]:
movies_df.fillna('', inplace = True)

# Split keywords and overview into their own dataframes for text cleaning

In [9]:
keywords_df = movies_df[['keywords']]
keywords_df.to_csv('keywords.csv', index=False)
keywords=pd.read_csv('keywords.csv')
keywords.head(4)

,keywords
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."


In [10]:
overview_df = movies_df[['overview']]
overview_df.to_csv('overview.csv', index=False)
overview_df=pd.read_csv('overview.csv')
overview_df.head(4)

,overview
0,"In the 22nd century, a paraplegic Marine is di..."
1,"Captain Barbossa, long believed to be dead, ha..."
2,A cryptic message from Bond’s past sends him o...
3,Following the death of District Attorney Harve...


# Text cleaning keywords: remove punctutation, then tokenize and lowercase 

In [11]:
# Display punctuation symbols
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
# Create function to remove punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

# Remove punctuation from data
keywords_df['keywords_w/o_punct'] = keywords_df['keywords'].apply(lambda x: remove_punct(x))
keywords_df.head()

/var/folders/cf/41c0zn6s2gd5fzv9l3rcsgr00000gn/T/ipykernel_2571/760405889.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywords_df['keywords_w/o_punct'] = keywords_df['keywords'].apply(lambda x: remove_punct(x))


,keywords,keywords_w/o_punct
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",id 1463 name culture clash id 2964 name future...
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",id 270 name ocean id 726 name drug abuse id 91...
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",id 470 name spy id 818 name based on novel id ...
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",id 849 name dc comics id 853 name crime fighte...
4,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",id 818 name based on novel id 839 name mars id...


In [13]:
# Create function to tokenize and lowercase data
def tokenize(text):
    # W+ means that either a word character (A-Za-z0-9_) or a dash (-) can go there.
    tokens = re.split('\W+', text)
    return tokens

# Tokenize and lowercase data 
keywords_df['tokenized_keywords'] = keywords_df['keywords_w/o_punct'].apply(lambda x: tokenize(x.lower())) 
keywords_df.head()

/var/folders/cf/41c0zn6s2gd5fzv9l3rcsgr00000gn/T/ipykernel_2571/2484443038.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywords_df['tokenized_keywords'] = keywords_df['keywords_w/o_punct'].apply(lambda x: tokenize(x.lower()))


,keywords,keywords_w/o_punct,tokenized_keywords
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",id 1463 name culture clash id 2964 name future...,"[id, 1463, name, culture, clash, id, 2964, nam..."
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",id 270 name ocean id 726 name drug abuse id 91...,"[id, 270, name, ocean, id, 726, name, drug, ab..."
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",id 470 name spy id 818 name based on novel id ...,"[id, 470, name, spy, id, 818, name, based, on,..."
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",id 849 name dc comics id 853 name crime fighte...,"[id, 849, name, dc, comics, id, 853, name, cri..."
4,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",id 818 name based on novel id 839 name mars id...,"[id, 818, name, based, on, novel, id, 839, nam..."


# Text cleaning overview: remove punctutation, tokenize and lowercase, stem, and remove stopwords 

In [14]:
overview_df.fillna('', inplace = True)

In [15]:
# Remove punctuation from data
overview_df['overview_w/o_punct'] = overview_df['overview'].apply(lambda x: remove_punct(x))
overview_df.head()

,overview,overview_w/o_punct
0,"In the 22nd century, a paraplegic Marine is di...",In the 22nd century a paraplegic Marine is dis...
1,"Captain Barbossa, long believed to be dead, ha...",Captain Barbossa long believed to be dead has ...
2,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...
3,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...
4,"John Carter is a war-weary, former military ca...",John Carter is a warweary former military capt...


In [16]:
# Tokenize and lowercase data 
overview_df['tokenized_overview'] = overview_df['overview_w/o_punct'].apply(lambda x: tokenize(x.lower())) 
overview_df.head()

,overview,overview_w/o_punct,tokenized_overview
0,"In the 22nd century, a paraplegic Marine is di...",In the 22nd century a paraplegic Marine is dis...,"[in, the, 22nd, century, a, paraplegic, marine..."
1,"Captain Barbossa, long believed to be dead, ha...",Captain Barbossa long believed to be dead has ...,"[captain, barbossa, long, believed, to, be, de..."
2,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...,"[a, cryptic, message, from, bond, s, past, sen..."
3,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...,"[following, the, death, of, district, attorney..."
4,"John Carter is a war-weary, former military ca...",John Carter is a warweary former military capt...,"[john, carter, is, a, warweary, former, milita..."


In [17]:
# Define stop words list
stopwords = nltk.corpus.stopwords.words('english')     # All English Stopwords
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [18]:
# Create function to remove stopwords
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords]
    return text

# Remove stop words from data
overview_df['stopwords_overview'] = overview_df['tokenized_overview'].apply(lambda x: remove_stopwords(x))
overview_df.head()

,overview,overview_w/o_punct,tokenized_overview,stopwords_overview
0,"In the 22nd century, a paraplegic Marine is di...",In the 22nd century a paraplegic Marine is dis...,"[in, the, 22nd, century, a, paraplegic, marine...","[22nd, century, paraplegic, marine, dispatched..."
1,"Captain Barbossa, long believed to be dead, ha...",Captain Barbossa long believed to be dead has ...,"[captain, barbossa, long, believed, to, be, de...","[captain, barbossa, long, believed, dead, come..."
2,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...,"[a, cryptic, message, from, bond, s, past, sen...","[cryptic, message, bond, past, sends, trail, u..."
3,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...,"[following, the, death, of, district, attorney...","[following, death, district, attorney, harvey,..."
4,"John Carter is a war-weary, former military ca...",John Carter is a warweary former military capt...,"[john, carter, is, a, warweary, former, milita...","[john, carter, warweary, former, military, cap..."


In [19]:
ps = nltk.PorterStemmer()

# Create function to apply stemmer
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

# Apply Porter Stemmer
overview_df['stemmed_overview'] = overview_df['stopwords_overview'].apply(lambda x: stemming(x))
overview_df.head()

,overview,overview_w/o_punct,tokenized_overview,stopwords_overview,stemmed_overview
0,"In the 22nd century, a paraplegic Marine is di...",In the 22nd century a paraplegic Marine is dis...,"[in, the, 22nd, century, a, paraplegic, marine...","[22nd, century, paraplegic, marine, dispatched...","[22nd, centuri, parapleg, marin, dispatch, moo..."
1,"Captain Barbossa, long believed to be dead, ha...",Captain Barbossa long believed to be dead has ...,"[captain, barbossa, long, believed, to, be, de...","[captain, barbossa, long, believed, dead, come...","[captain, barbossa, long, believ, dead, come, ..."
2,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...,"[a, cryptic, message, from, bond, s, past, sen...","[cryptic, message, bond, past, sends, trail, u...","[cryptic, messag, bond, past, send, trail, unc..."
3,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...,"[following, the, death, of, district, attorney...","[following, death, district, attorney, harvey,...","[follow, death, district, attorney, harvey, de..."
4,"John Carter is a war-weary, former military ca...",John Carter is a warweary former military capt...,"[john, carter, is, a, warweary, former, milita...","[john, carter, warweary, former, military, cap...","[john, carter, warweari, former, militari, cap..."


# Getting keywords from overview + merge with the keywords column to recreate the keywords column

In [20]:
# Create function to extract
def extract_key_words(input_str):
    r = Rake()
    r.extract_keywords_from_text(input_str.lower())
    key_words_dict_scores = r.get_word_degrees()
    sorted_key_words_dict_scores = sorted(key_words_dict_scores.items(), key=operator.itemgetter(1), reverse=True)
    sorted_dict = collections.OrderedDict(sorted_key_words_dict_scores)
#    return sorted_dict, list(sorted_dict.keys())[:round(len(sorted_dict.keys())/2)]
    return sorted_dict, list(sorted_dict.keys())[:5] # 5 is the number of the highest rated keywords returned
                                                         #  Change this value as you wish

In [21]:
overview_df['overview_keywords'] = overview_df['overview'].apply(lambda x: extract_key_words(x)[1]).apply(' '.join)
overview_df.head()

,overview,overview_w/o_punct,tokenized_overview,stopwords_overview,stemmed_overview,overview_keywords
0,"In the 22nd century, a paraplegic Marine is di...",In the 22nd century a paraplegic Marine is dis...,"[in, the, 22nd, century, a, paraplegic, marine...","[22nd, century, paraplegic, marine, dispatched...","[22nd, centuri, parapleg, marin, dispatch, moo...",22nd century paraplegic marine moon
1,"Captain Barbossa, long believed to be dead, ha...",Captain Barbossa long believed to be dead has ...,"[captain, barbossa, long, believed, to, be, de...","[captain, barbossa, long, believed, dead, come...","[captain, barbossa, long, believ, dead, come, ...",captain barbossa long believed come
2,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...,"[a, cryptic, message, from, bond, s, past, sen...","[cryptic, message, bond, past, sends, trail, u...","[cryptic, messag, bond, past, send, trail, unc...",bond terrible truth behind spectre
3,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...,"[following, the, death, of, district, attorney...","[following, death, district, attorney, harvey,...","[follow, death, district, attorney, harvey, de...",attorney gotham dent batman city
4,"John Carter is a war-weary, former military ca...",John Carter is a warweary former military capt...,"[john, carter, is, a, warweary, former, milita...","[john, carter, warweary, former, military, cap...","[john, carter, warweari, former, militari, cap...",carter former military captain reluctantly


In [22]:
overview_df['overviewv2'] = overview_df['overview_keywords'].apply(lambda x: tokenize(x.lower())) 
overview_df.head()

,overview,overview_w/o_punct,tokenized_overview,stopwords_overview,stemmed_overview,overview_keywords,overviewv2
0,"In the 22nd century, a paraplegic Marine is di...",In the 22nd century a paraplegic Marine is dis...,"[in, the, 22nd, century, a, paraplegic, marine...","[22nd, century, paraplegic, marine, dispatched...","[22nd, centuri, parapleg, marin, dispatch, moo...",22nd century paraplegic marine moon,"[22nd, century, paraplegic, marine, moon]"
1,"Captain Barbossa, long believed to be dead, ha...",Captain Barbossa long believed to be dead has ...,"[captain, barbossa, long, believed, to, be, de...","[captain, barbossa, long, believed, dead, come...","[captain, barbossa, long, believ, dead, come, ...",captain barbossa long believed come,"[captain, barbossa, long, believed, come]"
2,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...,"[a, cryptic, message, from, bond, s, past, sen...","[cryptic, message, bond, past, sends, trail, u...","[cryptic, messag, bond, past, send, trail, unc...",bond terrible truth behind spectre,"[bond, terrible, truth, behind, spectre]"
3,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...,"[following, the, death, of, district, attorney...","[following, death, district, attorney, harvey,...","[follow, death, district, attorney, harvey, de...",attorney gotham dent batman city,"[attorney, gotham, dent, batman, city]"
4,"John Carter is a war-weary, former military ca...",John Carter is a warweary former military capt...,"[john, carter, is, a, warweary, former, milita...","[john, carter, warweary, former, military, cap...","[john, carter, warweari, former, militari, cap...",carter former military captain reluctantly,"[carter, former, military, captain, reluctantly]"


In [23]:
overview_df['overviewv2'] = overview_df['overviewv2'].astype("string")
overview_df

,overview,overview_w/o_punct,tokenized_overview,stopwords_overview,stemmed_overview,overview_keywords,overviewv2
0,"In the 22nd century, a paraplegic Marine is di...",In the 22nd century a paraplegic Marine is dis...,"[in, the, 22nd, century, a, paraplegic, marine...","[22nd, century, paraplegic, marine, dispatched...","[22nd, centuri, parapleg, marin, dispatch, moo...",22nd century paraplegic marine moon,"['22nd', 'century', 'paraplegic', 'marine', 'm..."
1,"Captain Barbossa, long believed to be dead, ha...",Captain Barbossa long believed to be dead has ...,"[captain, barbossa, long, believed, to, be, de...","[captain, barbossa, long, believed, dead, come...","[captain, barbossa, long, believ, dead, come, ...",captain barbossa long believed come,"['captain', 'barbossa', 'long', 'believed', 'c..."
2,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...,"[a, cryptic, message, from, bond, s, past, sen...","[cryptic, message, bond, past, sends, trail, u...","[cryptic, messag, bond, past, send, trail, unc...",bond terrible truth behind spectre,"['bond', 'terrible', 'truth', 'behind', 'spect..."
3,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...,"[following, the, death, of, district, attorney...","[following, death, district, attorney, harvey,...","[follow, death, district, attorney, harvey, de...",attorney gotham dent batman city,"['attorney', 'gotham', 'dent', 'batman', 'city']"
4,"John Carter is a war-weary, former military ca...",John Carter is a warweary former military capt...,"[john, carter, is, a, warweary, former, milita...","[john, carter, warweary, former, military, cap...","[john, carter, warweari, former, militari, cap...",carter former military captain reluctantly,"['carter', 'former', 'military', 'captain', 'r..."
...,...,...,...,...,...,...,...
4797,El Mariachi just wants to play his guitar and ...,El Mariachi just wants to play his guitar and ...,"[el, mariachi, just, wants, to, play, his, gui...","[el, mariachi, wants, play, guitar, carry, fam...","[el, mariachi, want, play, guitar, carri, fami...",el mariachi guitar town case,"['el', 'mariachi', 'guitar', 'town', 'case']"
4798,A newlywed couple's honeymoon is upended by th...,A newlywed couples honeymoon is upended by the...,"[a, newlywed, couples, honeymoon, is, upended,...","[newlywed, couples, honeymoon, upended, arriva...","[newlyw, coupl, honeymoon, upend, arriv, respe...",newlywed couple respective sisters honeymoon,"['newlywed', 'couple', 'respective', 'sisters'..."
4799,"""Signed, Sealed, Delivered"" introduces a dedic...",Signed Sealed Delivered introduces a dedicated...,"[signed, sealed, delivered, introduces, a, ded...","[signed, sealed, delivered, introduces, dedica...","[sign, seal, deliv, introduc, dedic, quartet, ...",office dead letter seemingly undeliverable,"['office', 'dead', 'letter', 'seemingly', 'und..."
4800,When ambitious New York attorney Sam is sent t...,When ambitious New York attorney Sam is sent t...,"[when, ambitious, new, york, attorney, sam, is...","[ambitious, new, york, attorney, sam, sent, sh...","[ambiti, new, york, attorney, sam, sent, shang...",sam ambitious new york attorney,"['sam', 'ambitious', 'new', 'york', 'attorney']"


In [24]:
import json

In [25]:
MKeyword = keywords_df['keywords']
names4 = []
for movie in MKeyword:
    submovie = []
    movie = json.loads(movie)
    for attribute in movie: 
        submovie.append(attribute["name"])
    names4.append(submovie)
    for x in names4:
        json.dumps(x)

In [26]:
keywords_df.insert(3, 'keywordsv2', names4, True)

In [26]:
keywords_df['keywordsv2'] = keywords_df['keywordsv2'].astype("string")
keywords_df

C:\Users\mavic\AppData\Local\Temp/ipykernel_33044/1435373397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywords_df['keywordsv2'] = keywords_df['keywordsv2'].astype("string")


,keywords,keywords_w/o_punct,tokenized_keywords,keywordsv2
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",id 1463 name culture clash id 2964 name future...,"[id, 1463, name, culture, clash, id, 2964, nam...","['culture clash', 'future', 'space war', 'spac..."
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",id 270 name ocean id 726 name drug abuse id 91...,"[id, 270, name, ocean, id, 726, name, drug, ab...","['ocean', 'drug abuse', 'exotic island', 'east..."
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",id 470 name spy id 818 name based on novel id ...,"[id, 470, name, spy, id, 818, name, based, on,...","['spy', 'based on novel', 'secret agent', 'seq..."
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",id 849 name dc comics id 853 name crime fighte...,"[id, 849, name, dc, comics, id, 853, name, cri...","['dc comics', 'crime fighter', 'terrorist', 's..."
4,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",id 818 name based on novel id 839 name mars id...,"[id, 818, name, based, on, novel, id, 839, nam...","['based on novel', 'mars', 'medallion', 'space..."
...,...,...,...,...
4798,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",id 5616 name united statesu2013mexico barrier ...,"[id, 5616, name, united, statesu2013mexico, ba...","['united states–mexico barrier', 'legs', 'arms..."
4799,[],,[],[]
4800,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",id 248 name date id 699 name love at first sig...,"[id, 248, name, date, id, 699, name, love, at,...","['date', 'love at first sight', 'narration', '..."
4801,[],,[],[]


In [27]:
movies_df.drop(columns='keywords', inplace=True)
movies_df

,id,popularity,homepage,original_title,budget,revenue,genres,overview,cast,tagline
0,19995,150.437577,http://www.avatarmovie.com/,Avatar,237000000,2787965087,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",Enter the World of Pandora.
1,285,139.082615,http://disney.go.com/disneypictures/pirates/,Pirates of the Caribbean: At World's End,300000000,961000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","At the end of the world, the adventure begins."
2,206647,107.376788,http://www.sonypictures.com/movies/spectre/,Spectre,245000000,880674609,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",A Plan No One Escapes
3,49026,112.312950,http://www.thedarkknightrises.com/,The Dark Knight Rises,250000000,1084939099,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",The Legend Ends
4,49529,43.926995,http://movies.disney.com/john-carter,John Carter,260000000,284139100,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","Lost in our world, found in another."
...,...,...,...,...,...,...,...,...,...,...
4798,9367,14.269792,,El Mariachi,220000,2040920,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",El Mariachi just wants to play his guitar and ...,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","He didn't come looking for trouble, but troubl..."
4799,72766,0.642552,,Newlyweds,9000,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",A newlywed couple's honeymoon is upended by th...,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...",A newlywed couple's honeymoon is upended by th...
4800,231617,1.444476,http://www.hallmarkchannel.com/signedsealeddel...,"Signed, Sealed, Delivered",0,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...",
4801,126186,0.857008,http://shanghaicalling.com/,Shanghai Calling,0,0,[],When ambitious New York attorney Sam is sent t...,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...",A New Yorker in Shanghai


In [28]:
movies_df["keywords"] = keywords_df['keywordsv2'].astype(str) +"-"+ overview_df['overviewv2'].astype(str)
movies_df

,id,popularity,homepage,original_title,budget,revenue,genres,overview,cast,tagline,keywords
0,19995,150.437577,http://www.avatarmovie.com/,Avatar,237000000,2787965087,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",Enter the World of Pandora.,"['culture clash', 'future', 'space war', 'spac..."
1,285,139.082615,http://disney.go.com/disneypictures/pirates/,Pirates of the Caribbean: At World's End,300000000,961000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","At the end of the world, the adventure begins.","['ocean', 'drug abuse', 'exotic island', 'east..."
2,206647,107.376788,http://www.sonypictures.com/movies/spectre/,Spectre,245000000,880674609,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",A Plan No One Escapes,"['spy', 'based on novel', 'secret agent', 'seq..."
3,49026,112.312950,http://www.thedarkknightrises.com/,The Dark Knight Rises,250000000,1084939099,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",The Legend Ends,"['dc comics', 'crime fighter', 'terrorist', 's..."
4,49529,43.926995,http://movies.disney.com/john-carter,John Carter,260000000,284139100,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","Lost in our world, found in another.","['based on novel', 'mars', 'medallion', 'space..."
...,...,...,...,...,...,...,...,...,...,...,...
4798,9367,14.269792,,El Mariachi,220000,2040920,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",El Mariachi just wants to play his guitar and ...,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","He didn't come looking for trouble, but troubl...","['united states–mexico barrier', 'legs', 'arms..."
4799,72766,0.642552,,Newlyweds,9000,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",A newlywed couple's honeymoon is upended by th...,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...",A newlywed couple's honeymoon is upended by th...,"[]-['office', 'dead', 'letter', 'seemingly', '..."
4800,231617,1.444476,http://www.hallmarkchannel.com/signedsealeddel...,"Signed, Sealed, Delivered",0,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...",,"['date', 'love at first sight', 'narration', '..."
4801,126186,0.857008,http://shanghaicalling.com/,Shanghai Calling,0,0,[],When ambitious New York attorney Sam is sent t...,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...",A New Yorker in Shanghai,"[]-['brian', 'herzlinger', 'drew', 'barrymore'..."


# New feature 1 creation: actor names from the cast column of the credits dataset

In [29]:
movie_cast = movies_df['cast']
names2 = []
for movie in movie_cast:
    submovie = []
    movie = json.loads(movie)
    for attribute in movie: 
        submovie.append(attribute["name"])
    names2.append(submovie)

In [30]:
del movies_df['cast']
movies_df.insert(6, 'cast', names2, True)

In [31]:
movies_df['cast'] = movies_df['cast'].astype("string")
movies_df

,id,popularity,homepage,original_title,budget,revenue,cast,genres,overview,tagline,keywords
0,19995,150.437577,http://www.avatarmovie.com/,Avatar,237000000,2787965087,"['Sam Worthington', 'Zoe Saldana', 'Sigourney ...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,"['culture clash', 'future', 'space war', 'spac..."
1,285,139.082615,http://disney.go.com/disneypictures/pirates/,Pirates of the Caribbean: At World's End,300000000,961000000,"['Johnny Depp', 'Orlando Bloom', 'Keira Knight...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.","['ocean', 'drug abuse', 'exotic island', 'east..."
2,206647,107.376788,http://www.sonypictures.com/movies/spectre/,Spectre,245000000,880674609,"['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,"['spy', 'based on novel', 'secret agent', 'seq..."
3,49026,112.312950,http://www.thedarkknightrises.com/,The Dark Knight Rises,250000000,1084939099,"['Christian Bale', 'Michael Caine', 'Gary Oldm...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,The Legend Ends,"['dc comics', 'crime fighter', 'terrorist', 's..."
4,49529,43.926995,http://movies.disney.com/john-carter,John Carter,260000000,284139100,"['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","Lost in our world, found in another.","['based on novel', 'mars', 'medallion', 'space..."
...,...,...,...,...,...,...,...,...,...,...,...
4798,9367,14.269792,,El Mariachi,220000,2040920,"['Carlos Gallardo', 'Jaime de Hoyos', 'Peter M...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",El Mariachi just wants to play his guitar and ...,"He didn't come looking for trouble, but troubl...","['united states–mexico barrier', 'legs', 'arms..."
4799,72766,0.642552,,Newlyweds,9000,0,"['Edward Burns', 'Kerry Bishé', 'Marsha Dietle...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",A newlywed couple's honeymoon is upended by th...,A newlywed couple's honeymoon is upended by th...,"[]-['office', 'dead', 'letter', 'seemingly', '..."
4800,231617,1.444476,http://www.hallmarkchannel.com/signedsealeddel...,"Signed, Sealed, Delivered",0,0,"['Eric Mabius', 'Kristin Booth', 'Crystal Lowe...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...",,"['date', 'love at first sight', 'narration', '..."
4801,126186,0.857008,http://shanghaicalling.com/,Shanghai Calling,0,0,"['Daniel Henney', 'Eliza Coupe', 'Bill Paxton'...",[],When ambitious New York attorney Sam is sent t...,A New Yorker in Shanghai,"[]-['brian', 'herzlinger', 'drew', 'barrymore'..."


# New feature 2 creation: taking only the genre out of the genre column from the movies dataset 

In [32]:
movie_genre = movies_df['genres']
names3 = []
for movie in movie_genre:
    submovie = []
    movie = json.loads(movie)
    for attribute in movie: 
        submovie.append(attribute["name"])
    names3.append(submovie)
    for x in names3:
        json.dumps(x)

In [33]:
del movies_df['genres']
movies_df.insert(4, 'genres', names3, True)

In [34]:
movies_df['genres'] = movies_df['genres'].astype("string")
movies_df

,id,popularity,homepage,original_title,genres,budget,revenue,cast,overview,tagline,keywords
0,19995,150.437577,http://www.avatarmovie.com/,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",237000000,2787965087,"['Sam Worthington', 'Zoe Saldana', 'Sigourney ...","In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,"['culture clash', 'future', 'space war', 'spac..."
1,285,139.082615,http://disney.go.com/disneypictures/pirates/,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']",300000000,961000000,"['Johnny Depp', 'Orlando Bloom', 'Keira Knight...","Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.","['ocean', 'drug abuse', 'exotic island', 'east..."
2,206647,107.376788,http://www.sonypictures.com/movies/spectre/,Spectre,"['Action', 'Adventure', 'Crime']",245000000,880674609,"['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,"['spy', 'based on novel', 'secret agent', 'seq..."
3,49026,112.312950,http://www.thedarkknightrises.com/,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']",250000000,1084939099,"['Christian Bale', 'Michael Caine', 'Gary Oldm...",Following the death of District Attorney Harve...,The Legend Ends,"['dc comics', 'crime fighter', 'terrorist', 's..."
4,49529,43.926995,http://movies.disney.com/john-carter,John Carter,"['Action', 'Adventure', 'Science Fiction']",260000000,284139100,"['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...","John Carter is a war-weary, former military ca...","Lost in our world, found in another.","['based on novel', 'mars', 'medallion', 'space..."
...,...,...,...,...,...,...,...,...,...,...,...
4798,9367,14.269792,,El Mariachi,"['Action', 'Crime', 'Thriller']",220000,2040920,"['Carlos Gallardo', 'Jaime de Hoyos', 'Peter M...",El Mariachi just wants to play his guitar and ...,"He didn't come looking for trouble, but troubl...","['united states–mexico barrier', 'legs', 'arms..."
4799,72766,0.642552,,Newlyweds,"['Comedy', 'Romance']",9000,0,"['Edward Burns', 'Kerry Bishé', 'Marsha Dietle...",A newlywed couple's honeymoon is upended by th...,A newlywed couple's honeymoon is upended by th...,"[]-['office', 'dead', 'letter', 'seemingly', '..."
4800,231617,1.444476,http://www.hallmarkchannel.com/signedsealeddel...,"Signed, Sealed, Delivered","['Comedy', 'Drama', 'Romance', 'TV Movie']",0,0,"['Eric Mabius', 'Kristin Booth', 'Crystal Lowe...","""Signed, Sealed, Delivered"" introduces a dedic...",,"['date', 'love at first sight', 'narration', '..."
4801,126186,0.857008,http://shanghaicalling.com/,Shanghai Calling,[],0,0,"['Daniel Henney', 'Eliza Coupe', 'Bill Paxton'...",When ambitious New York attorney Sam is sent t...,A New Yorker in Shanghai,"[]-['brian', 'herzlinger', 'drew', 'barrymore'..."


# Vectorize Data

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'|\.")

In [44]:
movies_matrix = tfidf.fit_transform(movies_df['overview'].values.astype('U') + movies_df['keywords'] +movies_df['genres']+movies_df['cast'])
movies_matrix

AttributeError: 'NAType' object has no attribute 'lower'

# Application of cosine similarity

# Get recommendation based on cosine similarity